In [1]:
import functools
import numpy as np
import tensorflow as tf
from keras.datasets import cifar100
from keras import Sequential
from keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split
from keras.optimizers import SGD
from keras.layers import Dense, Activation, Dropout, Flatten, Conv2D, MaxPooling2D, LeakyReLU
from keras.layers.normalization import BatchNormalization
from tensorflow.keras.utils import to_categorical
from keras.utils.generic_utils import get_custom_objects
from keras.initializers import RandomNormal, GlorotNormal
from keras import regularizers

In [2]:
(x_train, y_train), (x_test, y_test) = cifar100.load_data()
x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size=.2)

169017344/169001437 [==============================] - 4s 0us/step


In [3]:
y_train = to_categorical(y_train)
y_val = to_categorical(y_val)
y_test = to_categorical(y_test)

In [4]:
x_train.shape, y_train.shape, x_val.shape, y_val.shape, x_test.shape, y_test.shape

((40000, 32, 32, 3),
 (40000, 100),
 (10000, 32, 32, 3),
 (10000, 100),
 (10000, 32, 32, 3),
 (10000, 100))

In [5]:
train_generator = ImageDataGenerator(rotation_range=2, horizontal_flip=True)
val_generator = ImageDataGenerator(rotation_range=2, horizontal_flip=True)
test_generator = ImageDataGenerator(rotation_range=2, horizontal_flip=True)

train_generator.fit(x_train)
val_generator.fit(x_val)
test_generator.fit(x_test)

In [6]:
def vgg16(act):
    model = Sequential()

    model.add(Conv2D(64, (3, 3), padding='same',
                        input_shape=(32, 32, 3), kernel_regularizer=regularizers.l2(0.0005)))
    model.add(Activation(act))
    model.add(BatchNormalization())
    model.add(Dropout(0.3))

    model.add(Conv2D(64, (3, 3), padding='same', kernel_regularizer=regularizers.l2(0.0005)))
    model.add(Activation(act))
    model.add(BatchNormalization())

    model.add(MaxPooling2D(pool_size=(2, 2)))

    model.add(Conv2D(128, (3, 3), padding='same', kernel_regularizer=regularizers.l2(0.0005)))
    model.add(Activation(act))
    model.add(BatchNormalization())
    model.add(Dropout(0.4))

    model.add(Conv2D(128, (3, 3), padding='same', kernel_regularizer=regularizers.l2(0.0005)))
    model.add(Activation(act))
    model.add(BatchNormalization())

    model.add(MaxPooling2D(pool_size=(2, 2)))

    model.add(Conv2D(256, (3, 3), padding='same', kernel_regularizer=regularizers.l2(0.0005)))
    model.add(Activation(act))
    model.add(BatchNormalization())
    model.add(Dropout(0.4))

    model.add(Conv2D(256, (3, 3), padding='same', kernel_regularizer=regularizers.l2(0.0005)))
    model.add(Activation(act))
    model.add(BatchNormalization())
    model.add(Dropout(0.4))

    model.add(Conv2D(256, (3, 3), padding='same', kernel_regularizer=regularizers.l2(0.0005)))
    model.add(Activation(act))
    model.add(BatchNormalization())

    model.add(MaxPooling2D(pool_size=(2, 2)))


    model.add(Conv2D(512, (3, 3), padding='same', kernel_regularizer=regularizers.l2(0.0005)))
    model.add(Activation(act))
    model.add(BatchNormalization())
    model.add(Dropout(0.4))

    model.add(Conv2D(512, (3, 3), padding='same', kernel_regularizer=regularizers.l2(0.0005)))
    model.add(Activation(act))
    model.add(BatchNormalization())
    model.add(Dropout(0.4))

    model.add(Conv2D(512, (3, 3), padding='same', kernel_regularizer=regularizers.l2(0.0005)))
    model.add(Activation(act))
    model.add(BatchNormalization())

    model.add(MaxPooling2D(pool_size=(2, 2)))


    model.add(Conv2D(512, (3, 3), padding='same', kernel_regularizer=regularizers.l2(0.0005)))
    model.add(Activation(act))
    model.add(BatchNormalization())
    model.add(Dropout(0.4))

    model.add(Conv2D(512, (3, 3), padding='same', kernel_regularizer=regularizers.l2(0.0005)))
    model.add(Activation(act))
    model.add(BatchNormalization())
    model.add(Dropout(0.4))

    model.add(Conv2D(512, (3, 3), padding='same', kernel_regularizer=regularizers.l2(0.0005)))
    model.add(Activation(act))
    model.add(BatchNormalization())

    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.5))

    model.add(Flatten())
    model.add(Dense(512, kernel_regularizer=regularizers.l2(0.0005)))
    model.add(Activation(act))
    model.add(BatchNormalization())

    model.add(Dropout(0.5))
    model.add(Dense(100))
    model.add(Activation('softmax'))

    return model

In [11]:
def gelu(x):
    return 0.5 * x * (1 + tf.tanh(tf.sqrt(2 / np.pi) * (x + 0.044715 * tf.pow(x, 3))))
get_custom_objects().update({'gelu': Activation(gelu)})

def swish(x):
    return x * tf.sigmoid(x)
get_custom_objects().update({'swish': Activation(swish)})

get_custom_objects().update({'leaky-relu': Activation(LeakyReLU(alpha=0.2))})

# act_func = ['tanh', 'relu', 'leaky-relu', 'elu', 'selu', 'gelu', 'swish']

model = vgg16('gelu')

batch_size = 128
epochs = 50

top3_acc = functools.partial(tf.keras.metrics.top_k_categorical_accuracy, k=3)

top3_acc.__name__ = 'top3_acc'

model.compile(optimizer=SGD(), loss='categorical_crossentropy', metrics=['accuracy', top3_acc, 'top_k_categorical_accuracy'])

history = model.fit(train_generator.flow(x_train, y_train, batch_size=batch_size), epochs=epochs,
                      validation_data=val_generator.flow(x_val, y_val, batch_size=batch_size), verbose=1)

Epoch 1/50
313/313 [==============================] - 45s 124ms/step - loss: 7.8841 - accuracy: 0.0111 - top3_acc: 0.0312 - top_k_categorical_accuracy: 0.0513 - val_loss: 6.8109 - val_accuracy: 0.0089 - val_top3_acc: 0.0319 - val_top_k_categorical_accuracy: 0.0521
Epoch 2/50
313/313 [==============================] - 37s 117ms/step - loss: 7.2872 - accuracy: 0.0171 - top3_acc: 0.0506 - top_k_categorical_accuracy: 0.0786 - val_loss: 6.7590 - val_accuracy: 0.0185 - val_top3_acc: 0.0475 - val_top_k_categorical_accuracy: 0.0772
Epoch 3/50
313/313 [==============================] - 37s 120ms/step - loss: 6.9972 - accuracy: 0.0298 - top3_acc: 0.0801 - top_k_categorical_accuracy: 0.1229 - val_loss: 6.6728 - val_accuracy: 0.0230 - val_top3_acc: 0.0579 - val_top_k_categorical_accuracy: 0.0986
Epoch 4/50
313/313 [==============================] - 36s 116ms/step - loss: 6.8520 - accuracy: 0.0372 - top3_acc: 0.0980 - top_k_categorical_accuracy: 0.1495 - val_loss: 6.5896 - val_accuracy: 0.0277 - va

In [12]:
print(history.history)
# y_pred = np.argmax(model.predict(x_test), axis=1)
# y_true = np.argmax(y_test,axis=1)

# print(y_pred.shape)
# print(y_true.shape)

# print(np.sum(y_pred == y_true) / y_pred.shape[0])

print(model.evaluate(x_test, y_test))

{'loss': [7.67201566696167, 7.2097063064575195, 6.9503607749938965, 6.8054351806640625, 6.675176620483398, 6.57338285446167, 6.493958950042725, 6.403574466705322, 6.3328776359558105, 6.268405437469482, 6.196023464202881, 6.131565570831299, 6.0729169845581055, 6.001755714416504, 5.976963043212891, 5.913079261779785, 5.841590881347656, 5.770167350769043, 5.733740329742432, 5.663923263549805, 5.610350608825684, 5.53655481338501, 5.490584850311279, 5.430843830108643, 5.363954067230225, 5.310644626617432, 5.2514543533325195, 5.196191310882568, 5.140029430389404, 5.0798516273498535, 5.035109519958496, 4.9788432121276855, 4.918978214263916, 4.860324382781982, 4.838574409484863, 4.768493175506592, 4.703679084777832, 4.65272331237793, 4.595791816711426, 4.54779052734375, 4.501707553863525, 4.4523797035217285, 4.403788089752197, 4.336930751800537, 4.279697418212891, 4.231776714324951, 4.185236930847168, 4.126057147979736, 4.086338996887207, 4.047345161437988], 'accuracy': [0.012024999596178532, 